In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob



CHECKERBOARD = (7,7)

points3dL = []
points2dL = []

points3dR = []
points2dR = []

objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

chessboard_images_left = []
chessboard_images_right = []
for i in range(1,5):
    chessboard_images_left.append(cv2.imread('chessboards/c' + str(i) + 'Left.png'))
    chessboard_images_right.append(cv2.imread('chessboards/c' + str(i) + 'Right.png'))

for img in chessboard_images_left:

    grayL = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(grayL, CHECKERBOARD, None)
    if ret:
        #img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        points3dL.append(objp)
        points2dL.append(corners)

for img in chessboard_images_right:

    grayR = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(grayR, CHECKERBOARD, None)
    if ret:
        #img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners, ret)
        points3dR.append(objp)
        points2dR.append(corners)
    #cv2.imshow('img',img)
    #cv2.waitKey(0)

# cv2.destroyAllWindows()


 

h,w = img.shape[:2]
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(points3dL, points2dL, grayL.shape[::-1], None, None)


opticalCenterL= np.array(mtxL[0][2], mtxL[1][2]) #cx, cy
focalLengthL = np.array(mtxL[0][0], mtxL[1][1]) #fx, fy


retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(points3dR, points2dR, grayR.shape[::-1], None, None)

opticalCenterR= np.array(mtxR[0][2], mtxR[1][2]) #cx, cy
focalLengthR = np.array(mtxR[0][0], mtxR[1][1]) #fx, fy

print("here")
print(rvecsR)
print(rvecsL)
print(rvecsL-rvecsR)


#stereo calibration using the left and right camera matrices and distortion coefficients
ret, mtxL, distL, mtxR, distR, R, T, E, F = cv2.stereoCalibrate(points3dL, points2dL, points2dR, mtxL, distL, mtxR, distR, grayL.shape[::-1]) #", None, None, None, None, cv2.CALIB_FIX_INTRINSIC, (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))"


# optical centre = inv(rot)*-trans

# maybe use triangulate point



#use stereoRectify to calculate the rectification transforms
rectify_scale = 0 #alpha
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(mtxL, distL, mtxR, distR, grayL.shape[::-1], R, T, rectify_scale, (0,0)) # last paramater is alpha, if 0= crop, if 1= all pixels

print(F)



(array([[-0.63212731],
       [-0.86623647],
       [-0.27923931]]), 
       array([[-0.66821302],
       [-0.47988929],
       [-0.16349573]]), 
       array([[0.19554645],
       [0.50652313],
       [1.55886446]]), 
       array([[0.51365943],
       [0.8107809 ],
       [1.50485785]]), 
       array([[-0.06132242],
       [-0.18139892],
       [ 0.07797375]]), 
       array([[-0.0465299 ],
       [ 0.21514505],
       [ 0.06877913]]))


(array([[-0.66840644],
       [-0.47962099],
       [-0.16328374]]), 
       array([[0.51390699],
       [0.8112565 ],
       [1.50470384]]), 
       array([[-0.04659729],
       [ 0.21604998],
       [ 0.06878314]]))
(array([[-0.6321246 ],
       [-0.86663674],
       [-0.27947595]]), 
       array([[0.19511143],
       [0.50613904],
       [1.55891877]]), 
       array([[-0.06152   ],
       [-0.18140996],
       [ 0.07795007]]))

here
(array([[-0.66840644],
       [-0.47962099],
       [-0.16328374]]), array([[0.51390699],
       [0.8112565 ],
       [1.50470384]]), array([[-0.04659729],
       [ 0.21604998],
       [ 0.06878314]]))
(array([[-0.6321246 ],
       [-0.86663674],
       [-0.27947595]]), array([[0.19511143],
       [0.50613904],
       [1.55891877]]), array([[-0.06152   ],
       [-0.18140996],
       [ 0.07795007]]))


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [18]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import math

skip = 10


F = np.array([[-3.93215154e-08,  7.82607265e-07, -1.96522540e-04],[ 1.05197628e-06,  4.40299094e-08, -1.24505789e-02],[-8.53593685e-04 , 1.07493969e-02 , 1.00000000e+00]])

chimp_left = []
chimp_right = []
chimp_left_b = []
chimp_right_b = []

chimp_left = glob.glob('scanLeft/*.png')
chimp_right = glob.glob('scanRight/*.png')

for fname in chimp_left:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    chimp_left_b.append(cv2.inRange(gray,30,160))

for fname in chimp_right:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    chimp_right_b.append(cv2.inRange(gray,30,160))


#print(chimp_left_b[11])

#----------------------------------Test F
imgt = chimp_left_b[11]
imgtr = cv2.imread(chimp_right[11])
imgr = cv2.imread(chimp_left[11])
#cv2.imshow('img', imgt)
# cv2.waitKey(0)

# print(imgt.shape)
# imgr[imgt==255] = [0,255,0]
ind = np.where(imgt == 255)
# print(ind)

i=0
for y,x in zip(ind[0],ind[1]):
    if i ==200:
        lp = F@np.array([[x],[y],[1]])
        start = -(lp[0]*100+lp[2])/lp[1]
        stop= -(lp[0]*1200+lp[2])/lp[1]
        imgtr = cv2.line(imgtr, (100,int(start)), (1200,int(stop)), (0,255,0), 1)
        i = 0
    i+=1



#-----------------------------------------


#------------------------------------test f cv2
imgl = chimp_left_b[11]
imgr = chimp_right_b[11]
imgtr = cv2.imread(chimp_right[11])
imgtl = cv2.imread(chimp_left[11])

pointsLeft = np.argwhere(imgl == 255)
pointsRight = np.argwhere(imgr == 255)

# print("shape")
# print(pointsLeft.shape[0])
# print(pointsRight.shape[0])
# pointsLeft = pointsLeft[0:-1:math.ceil(pointsLeft.shape[0]/50)]
# pointsRight = pointsRight[0:-1:math.ceil(pointsRight.shape[0]/50)]
# print(pointsLeft.shape)

reducedLeft = {}
nb = {}
for i in range(len(pointsLeft)):
    pointLeft = pointsLeft[i]
    if pointLeft[0] in reducedLeft.keys():
        reducedLeft[pointLeft[0]] += pointLeft[1]
        nb[pointLeft[0]] += 1
    else:
        reducedLeft[pointLeft[0]] = pointLeft[1]
        nb[pointLeft[0]] = 1

for key in reducedLeft.keys():
    reducedLeft[key] = int(reducedLeft[key]/nb[key])

print(len(reducedLeft))

#pointsLeft = np.argwhere(imgl == 255)


#-------------------------------------------------------------------
# left_points = np.array([[j,i] for i,j in reducedLeft.items()])
# right_points = best_points

# # Matrices de rotation (s'obtient à partir du vecteur de rotation avec Rodrigues) :
# rotRgt = cv2.Rodrigues(rvecsR[2])[0]
# rotLft = cv2.Rodrigues(rvecsL[2])[0]

# # On compose la matrice extrinsèque en combinant les matrices de rotation et le vecteur de translation (3x4) :
# extRgt = np.concatenate((rotRgt,tvecsR[2]), axis=1)
# extLft = np.concatenate((rotLft,tvecsL[2]), axis=1)

# # On récupère la matrice de projection avec matrice intrinsèqe @ matrice extrinsèque :
# projRgt = mtxR @ extRgt 
# projLft = mtxL @ extLft

# centerRgt = np.concatenate(((-1*np.transpose(rotRgt)) @ tvecsR[2], [[1]]), axis = 0)
# centerLft = np.concatenate(((-1*np.transpose(rotLft)) @ tvecsL[2], [[1]]), axis = 0)


# # Skew symmetric matrix
# def skew(v):
#     v = v[:,0]
#     return np.array([[  0 ,-v[2], v[1]],
#                     [ v[2],   0 ,-v[0]],
#                     [-v[1], v[0],  0  ]])

# # On prend le cross product [P'C]x @ la matrice de projection P' @ la pseudo-inverse P+

# fund = np.array(skew(projLft @ centerRgt) @ projLft @ np.linalg.pinv(projRgt)) 
#---------------------------------------------------------


k=0

lines = []
for i,j in reducedLeft.items():
    #pointLeft = pointsLeft[j]
    pointLeft = np.array([j, i])
    pointLeft = np.array([pointLeft], dtype=np.float32)
    line = cv2.computeCorrespondEpilines(pointLeft, 1, F)
    line = line.reshape(-1,3)
    #print(lines)
    lines.append(line)

    #draw epiline on right image in white color and display it
    x0, y0 = map(int, [0, -line[0][2]/line[0][1] ])
    x1, y1 = map(int, [imgtr.shape[1], -(line[0][2]+line[0][0]*imgtr.shape[1])/line[0][1] ])
    if k ==skip:
        imgtr = cv2.line(imgtr, (x0, y0), (x1, y1), (0, 255, 0), 1)
        k = 0

    k+=1


print(len(lines))



reducedRight = {}
nb = {}
for i in range(len(pointsRight)):
    pointRight = pointsRight[i]
    #pointRight = np.array([pointRight[1], pointRight[0]])
    if pointRight[0] in reducedRight.keys():
        reducedRight[pointRight[0]] += pointRight[1]
        nb[pointRight[0]] += 1
    else:
        reducedRight[pointRight[0]] = pointRight[1]
        nb[pointRight[0]] = 1

for key in reducedRight.keys():
    reducedRight[key] = int(reducedRight[key]/nb[key])

for i,j in reducedRight.items():
    imgtr[i,j] = [0,255,0]

# cv2.imshow('img', imgtr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

#compute distance between a line and a point
def distPointLine(point, line):
    return abs(line[0]*point[0]+line[1]*point[1]+line[2])/np.sqrt(line[0]**2+line[1]**2)

best_points=[]
for line in lines:
    # print(line)
    line = line[0]
    minDist = float("inf")
    for i,j in reducedRight.items():
        point = np.array([j, i])
        dist = distPointLine(point, line)
        if dist < minDist:
            minDist = dist
            best_point = point
    #print(best_point)
    #if minDist != float("inf"):
    best_points.append(best_point)

best_points = np.array(best_points, dtype=np.int32)
print(best_points[0])
k=0

for elem in best_points:
    if k ==skip:
        imgtr[elem[1],elem[0]] = [255,0,255]
        for i in range(-3,3):
            imgtr[elem[1],elem[0]+i] = [255,0,255]

        k = 0
    k+=1
#imgtr[best_points[0][0],best_points[0][1]] = [0,0,255]
# cv2.imshow('img', imgtr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()





698
698
[1038  141]


In [33]:
left_points = np.array([[j,i] for i,j in reducedLeft.items()])
right_points = best_points

# Matrices de rotation (s'obtient à partir du vecteur de rotation avec Rodrigues) :
rotRgt = cv2.Rodrigues(rvecsR[2])[0]
rotLft = cv2.Rodrigues(rvecsL[2])[0]

centerLft = np.linalg.inv(rotLft) @ -tvecsL[2]
centerRgt = np.linalg.inv(rotRgt) @ -tvecsR[2]
print(centerLft)

camCenterLeft = np.transpose(centerLft)[0]
camCenterRight = np.transpose(centerRgt)[0]
print(camCenterLeft)


from mathutils import geometry as pygeo
from mathutils import Vector


def get_intersection(left_cam, right_cam, left_world, right_world):

    leftCam = Vector((camCenterLeft[0],camCenterLeft[1],camCenterLeft[2]))
    rightCam = Vector((camCenterRight[0],camCenterRight[1],camCenterRight[2]))

    leftWorld = Vector((worldL[0],worldL[1],worldL[2]))
    rightWorld = Vector((worldR[0],worldR[1],worldR[2]))

    intersection = pygeo.intersect_line_line(leftCam,leftWorld,rightCam,rightWorld)
    return intersection

Point_3D = []
print()
print(left_points[1][0])
print(left_points[1][1])
# for i in range(left_points.shape[0]):
#     worldL = np.array([left_points[i][0],left_points[i][1],0])
#     worldR = np.array([right_points[i][0],right_points[i][1],0])
#     Point_3D.append(get_intersection(camCenterLeft,camCenterRight,worldL,worldR))


[[ -0.95201288]
 [  4.83220057]
 [-19.66046928]]
[ -0.95201288   4.83220057 -19.66046928]
1141
129


In [ ]:
# print("best")
# print(best_points[0:5])
# print(best_points.T.shape)
# print("left")
# items = reducedLeft.items()
#test = np.array(list(reducedLeft.items()))
test = []
for i, j in reducedLeft.items():
    test.append([j, i])
test = np.array(test, dtype = np.float64)
# print(test.T.shape)

test2 = test[0:-1:math.ceil(test.shape[0]/50)]
best_points2 = best_points[0:-1:math.ceil(best_points.shape[0]/50)]

# print(test2)
# print("dddddddddddddd")
# print(best_points2)

# print(test2[0:3])
# print(best_points2[0:3])
# print(test2.T.shape)
# print(best_points2.shape)
# print(P1)




# #-----------------------------------------Triangulation
ul = cv2.undistortPoints(test2, mtxL, distL)
ur = cv2.undistortPoints(best_points2, mtxR, distR)

res = cv2.triangulatePoints(P1, P2, ul.T, ur.T)
# print(res.shape)
# print(res[0])
#res = res/res[3]
# res = res.T
# print(res[0])


fig = plt.figure(figsize=(4,4))

ax = fig.add_subplot(111, projection='3d')

for elem in res.T:
    if elem[3] == 0:
        continue
    elem = elem/elem[3]
    print(elem)
    ax.scatter(elem[0], elem[1], elem[2], c='r', marker='o')
plt.show()